In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

path = # your path here

In [ ]:
expected_answers = ['respectful', 'toxic']
def get_misbehavior_rate(model_answer):
    if model_answer in expected_answers:
        return "Expected"
    else:
        return "Unexpected"

In [ ]:
def hallucination_rate(preds):
  """
  returns the percentage of hallucinated sentences considering reference and counterfactual sentences.

  @params:
  preds: df with 'results_1' and 'results_2' columns
  """

  preds['misbehavior_1'] = [get_misbehavior_rate(output) for output in preds.results_1]
  preds['misbehavior_2'] = [get_misbehavior_rate(output) for output in preds.results_2]

  n_unexpected = preds[(preds['misbehavior_1'] == 'Unexpected') | (preds['misbehavior_2'] == 'Unexpected')].shape[0]

  return (n_unexpected / preds.shape[0])*100

In [ ]:
def hallucination_rate_counterfactuals_wrt_original(preds):
  """
  returns the percentage of hallucinated sentences generated by the model when
  it sees the counterfactuals but does not hallucinate when it sees the original sentences.

  @params:
  preds: df with 'results_1' (counterfactual) and 'results_2' (original) columns
  """

  preds['misbehavior_1'] = [get_misbehavior_rate(output) for output in preds.results_1]
  preds['misbehavior_2'] = [get_misbehavior_rate(output) for output in preds.results_2]

  n_unexpected = preds[(preds['misbehavior_1'] == 'Unexpected') & (preds['misbehavior_2'] == 'Expected')].shape[0]

  return (n_unexpected / preds.shape[0])*100

In [ ]:
def hallucination_rate_counterfactuals_only(preds):
  """
  returns the percentage of hallucinated sentences generated by the model when it sees the counterfactuals.

  @params:
  preds: df with 'results_1' (counterfactual) and 'results_2' (original) columns
  """

  preds['misbehavior_1'] = [get_misbehavior_rate(output) for output in preds.results_1]
  preds['misbehavior_2'] = [get_misbehavior_rate(output) for output in preds.results_2]

  n_unexpected = preds[preds['misbehavior_1'] == 'Unexpected'].shape[0]

  return (n_unexpected / preds.shape[0])*100

In [ ]:
def label_flip_rate(preds):
  """
  returns the percentage of label flips.

  @params:
  preds: df with 'results_1' and 'results_2' columns
  """

  preds['misbehavior_1'] = [get_misbehavior_rate(output) for output in preds.results_1]
  preds['misbehavior_2'] = [get_misbehavior_rate(output) for output in preds.results_2]

  expected_rows = preds[(preds['misbehavior_1'] == 'Expected') & (preds['misbehavior_2'] == 'Expected')]
  num_label_flips = expected_rows[expected_rows['results_1'] != expected_rows['results_2']].shape[0]
  total_expected = expected_rows.shape[0]

  return (num_label_flips / total_expected) * 100

In [ ]:
def f1_score(preds):
  """
  returns the f1-score.

  @params:
  preds: dataframe with 'results_1' and 'results_2' columns -- we only need 'results_2' because those are the non-counterfactual sentences
  """

  preds['misbehavior_2'] = [get_misbehavior_rate(output) for output in preds.results_2]

  df = pd.read_excel(path+'data/inputs experiment 2.xlsx')
  df['Label'].replace({'hateful': 'toxic', 'non-hateful':'respectful'}, inplace=True)

  df['Predicted'] = preds['results_2']
  df['misbehavior'] = preds['misbehavior_2']
  df = df[df['misbehavior']=='Expected']
  df.reset_index(drop=True, inplace=True)

  true_labels = df['Label'].to_list()
  predicted_labels = df['Predicted'].to_list()

  print(classification_report(true_labels, predicted_labels, zero_division=False, digits=4))